In [6]:
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from sklearn.decomposition import PCA
#import matplotlib.pyplot as plt
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import pyarrow
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import pypdf 

In [2]:
os.getcwd()

'/Users/hyunjikim/Desktop/UChicago/Gen AI/UChicagoADSRagAssistant'

## Loading articles

In [7]:
pdf_paths = [
    "4_ms_ads_all_pages.pdf"
]

# Load and split documents
loaders = [PyPDFLoader(path) for path in pdf_paths]
docs = []
for loader in loaders:
    docs.extend(loader.load())

## Chunking Data

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,     # Max characters per chunk
    chunk_overlap=100    # Overlap between chunks to preserve context
)

In [9]:
chunks = splitter.split_documents(docs)
print(f"Total chunks: {len(chunks)}")

Total chunks: 274


In [10]:
chunks[1]

Document(metadata={'producer': 'PDFCreator Online (www.pdfforge.org/online)', 'creator': 'PDFCreator Online (www.pdfforge.org/online)', 'creationdate': '2025-05-18T21:43:39+00:00', 'title': 'Merged with PDFCreator Online', 'moddate': '2025-05-18T21:43:40+00:00', 'source': '4_ms_ads_all_pages.pdf', 'total_pages': 41, 'page': 0, 'page_label': '1'}, page_content='healthcare, and business.\nThe career outcomes for our MS in Applied Data Science graduates are exceptional. Explore the diverse career paths our alumni have pursued, from top employers to leading\nindustries actively seeking our graduates.\nPage:https://datascience.uchicago.edu/education/mprograms/ms-in-applied-data-science/online-')

## Storing in Vector Database

In [19]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [12]:
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db")

## Example pass through of 1 query

In [14]:
# Define a new query
query = "Who is the director of the program"


retriever = db.as_retriever(search_kwargs={"k": 3})
retrieved_docs = retriever.get_relevant_documents(query)
context = "\n".join([doc.page_content for doc in retrieved_docs])
rag_prompt = f"""You are a helpful assistant. Use the following context to answer the question.

[CONTEXT]
{context}

[QUESTION]
{query}

Answer:"""

print("=== RAG Augmented Prompt ===")
print(rag_prompt)

=== RAG Augmented Prompt ===
You are a helpful assistant. Use the following context to answer the question.

[CONTEXT]
Chicago the expectations for technical competence, communication and influence skills, and exposure to advanced Data Science evolving technologies is high. The skills learned in
the program translate directly into practice due to the program’s balance between theory and rigorous application experience developed in coursework and the Capstone project work
delivered across the curriculum.
Is the Master's in Applied Data Science Online program equally academically rigorous as the In-Person program?Yes. TheOnline Programcurriculum is overseen by the same faculty
curriculum committee as the In-Person program. Both programs are jointly reviewed and are held to the same high standards. Additionally, both programs are granted by the
University of ChicagoPhysical Sciences Division.
Yes. TheOnline Programcurriculum is overseen by the same faculty curriculum committee as the In-P

## Loading LLM Model

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline

model_name = "google/flan-t5-base"  

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create inference pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use mps:0


In [18]:
# Run generation
response = llm(rag_prompt)
print("\n=== Generated Answer ===")
print(response)


=== Generated Answer ===
Patrick Vonesh


## Retriever 2

### Question Answer Dataset Generated Through 'YourBench' Mistral 'mistralai/Mixtral-8x7B-Instruct-v0.1'

https://yourbench.github.io/

In [21]:
df = pd.read_parquet("train-00000-of-00001.parquet")

In [22]:
df.head()

,chunk_id,document_id,additional_instructions,question,self_answer,choices,estimated_difficulty,self_assessed_question_type,generating_model,thought_process,raw_response,citations
0,768bc379-8ce4-466d-bf0d-398c888911dd_0,768bc379-8ce4-466d-bf0d-398c888911dd,undergraduate,What industries have shown significant interes...,"Industries such as technology, finance, health...",[],3,factual,mistralai/Mixtral-8x7B-Instruct-v0.1,Identify the industries where MS-ADS graduates...,<document_analysis>\n\nThe text chunk is a br...,[employment across industries such as technolo...
1,768bc379-8ce4-466d-bf0d-398c888911dd_0,768bc379-8ce4-466d-bf0d-398c888911dd,undergraduate,What percentage growth in employment is expect...,The employment for data scientists is projecte...,[],3,factual,mistralai/Mixtral-8x7B-Instruct-v0.1,Understand the projected growth rate for data ...,<document_analysis>\n\nThe text chunk is a br...,[employment for data scientists is projected t...
2,768bc379-8ce4-466d-bf0d-398c888911dd_0,768bc379-8ce4-466d-bf0d-398c888911dd,undergraduate,What is the primary reason for the surge in de...,The increasing reliance on data science across...,[],4,conceptual,mistralai/Mixtral-8x7B-Instruct-v0.1,Recognize the main driver for the growing dema...,<document_analysis>\n\nThe text chunk is a br...,[This surge is driven by the increasing relian...
3,768bc379-8ce4-466d-bf0d-398c888911dd_1,768bc379-8ce4-466d-bf0d-398c888911dd,undergraduate,What advantages does the flexibility of the on...,The flexibility of the online MS-ADS program a...,[],5,conceptual,mistralai/Mixtral-8x7B-Instruct-v0.1,"Considering the text chunk, I wanted to create...",<document_analysis>\nThe text chunk provided ...,"[You will benefit fromthe same top faculty, br..."
4,768bc379-8ce4-466d-bf0d-398c888911dd_1,768bc379-8ce4-466d-bf0d-398c888911dd,undergraduate,What does the term 'rigor' imply in the contex...,"In the context of the online MS-ADS program, '...",[],6,analytical,mistralai/Mixtral-8x7B-Instruct-v0.1,As the text highlights the rigor of the online...,<document_analysis>\nThe text chunk provided ...,"[You will benefit fromthe same top faculty, br..."


In [23]:
df['finalchunk']='question: '+df['question']+'\n' +'answer: '+df['self_answer']

In [24]:
chunks2=list(df['finalchunk'])
chunks2[1]

'question: What percentage growth in employment is expected for data scientists from 2023 to 2033?\nanswer: The employment for data scientists is projected to grow by 36% from 2023 to 2033.'

In [25]:
documents = [Document(page_content=text) for text in chunks2]

# Now call Chroma correctly
db2 = Chroma.from_documents(documents, embedding_model, persist_directory="./chroma_db")

## Running all queries

In [26]:
queries = [
    'What is the deadline to apply?',
    'What is time series course about?',
    'What happens in capstone?',
    'Where are the classes held?',
    'which companies do alumni work?',
    "What's the difference between in-person and online program?",
    "Who is the director of the program?",
    "Tell me about the France"]

In [ ]:
for query in queries:
    retriever = db.as_retriever(search_kwargs={"k": 3})
    retrieved_docs = retriever.get_relevant_documents(query)
    
    # Combine the retrieved context into a single block of text
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    
    retriever2 = db2.as_retriever(search_kwargs={"k": 3})
    retrieved_docs2 = retriever2.get_relevant_documents(query)
    
    # Combine the retrieved context into a single block of text
    context2 = "\n".join([doc.page_content for doc in retrieved_docs2])
    
    # Compose the RAG prompt by combining the context and the user's question
    rag_prompt = f"""You are a helpful assistant. Use the following context to answer the question related to the MS Applied Data Science Program. If you don't know the answer, just say to reach out to contact of the program.
    
    [CONTEXT]
    {context}

    [CONTEXT2]
    {context2}
    
    [QUESTION]
    {query}
    
    Answer:"""

    response = llm(rag_prompt)
    print('-'*100)
    print('QUERY-----------------------------')
    print(query)
    #print('PROMPT----------------------------')
    #print(rag_prompt)
    print('ANSWER:---------------------------')
    print(response)

----------------------------------------------------------------------------------------------------
QUERY-----------------------------
What is the deadline to apply?
ANSWER:---------------------------
May 6, 2025 – Third Priority Application Deadline, June 23, 2025 – Final Application Deadline.
----------------------------------------------------------------------------------------------------
QUERY-----------------------------
What is time series course about?
ANSWER:---------------------------
Time Series Analysis and Forecasting
----------------------------------------------------------------------------------------------------
QUERY-----------------------------
What happens in capstone?
ANSWER:---------------------------
The capstone project, completed over two quarters, covers research design, implementation, and writing, preparing students for exceptional career outcomes in data science.
--------------------------------------------------------------------------------------------